In [1]:
import requests
import json
import os
import cgi
import pandas as pd

This code was developped following the great example provided by Yaw Anokwa (ODK)
https://forum.getodk.org/t/error-in-get-request-for-central-api/26571/10

### Session authentication

https://odkcentral.docs.apiary.io/#reference/authentication/session-authentication/logging-in

In [2]:
def get_session_token(central_url, central_email, central_password):

    email_token_response = requests.post(
        central_url + "/v1/sessions",
        data = json.dumps({"email": central_email, "password": central_password}),
        headers = {"Content-Type": "application/json"},
    )

    if email_token_response.status_code == 200:
        return email_token_response.json()["token"]

### List projects

https://odkcentral.docs.apiary.io/#reference/project-management/projects/listing-projects

In [3]:
def list_projects(central_url, session_token):

    projects_response = requests.get(
        central_url + "/v1/projects/",
        headers={"Authorization": "Bearer " + session_token},
    )

    projects = {}
    for project in projects_response.json():
        projects[project["id"]] = project["name"]

    return projects

### List app users

https://odkcentral.docs.apiary.io/#reference/accounts-and-users/users/listing-all-app-users

In [4]:
def list_app_users(central_url, session_token, central_project_id):

    app_users_response = requests.get(
        central_url + "/v1/projects/" + str(central_project_id) + "/app-users",
        headers={"Authorization": "Bearer " + session_token},
    )

    app_users = {}
    for app_user in app_users_response.json():
        app_users[app_user["id"]] = app_user["displayName"]
    return app_users

### Listing all submissions on a form

https://odkcentral.docs.apiary.io/#reference/forms-and-submissions/submissions/listing-all-submissions-on-a-form

In [5]:
def list_all_submissions(central_url, session_token, central_project_id, central_form_id):

    submission_response = requests.get(
        central_url + "/v1/projects/" + str(central_project_id) + "/forms/" + str(central_form_id) + "/submissions",
        headers={"Authorization": "Bearer " + session_token},
    )
    
    if submission_response.status_code == 200:
        d = submission_response.json()
        return pd.DataFrame(d)

### Odata Endpoints Data Document

https://odkcentral.docs.apiary.io/#reference/odata-endpoints/odata-form-service/data-document

In [6]:
def data_document(central_url, session_token, central_project_id, central_form_id):

    data_response = requests.get(
        central_url + "/v1/projects/" + str(central_project_id) + "/forms/" + str(central_form_id) + ".svc/Submissions",
        headers={"Authorization": "Bearer " + session_token},
    )

    if data_response.status_code == 200:
        d = data_response.json()['value']
        return pd.DataFrame(d)

### Direct Backup using an ad-hoc passphrase

https://odkcentral.docs.apiary.io/#reference/system-endpoints/direct-backup

In [7]:
def backup(central_url, session_token, passphrase):

    backup_response = requests.post(
        central_url + "/v1/backup",
        data = json.dumps({"passphrase": passphrase}),
        headers = {"Authorization": "Bearer " + session_token,
                   "Content-Type": "application/json"},
        stream = True
    )

    if backup_response.status_code == 200:
        _, params = cgi.parse_header(backup_response.headers.get('Content-Disposition', ''))
        filename = params["filename"].replace(":","-")
        file = open(filename, "wb")
        file.write(backup_response.content)
        print("Backup successful")

### Main

In [8]:
with open("odk_central_credentials.json") as f:
    credentials = json.load(f)
    central_url = credentials["central_url"]
    central_email = credentials["central_email"]
    central_password = credentials["central_password"]

    session_token = get_session_token(central_url, central_email, central_password)
    if session_token:
        # List all projects
        projects = list_projects(central_url, session_token)
        # For each project, list all app users
        for project_id, project_name in projects.items():
            print("- Project", project_id, "- " + project_name)
            app_users = list_app_users(central_url, session_token, project_id)
            for app_user_id, app_user_name in app_users.items():
                print("  - App user", app_user_id, "- " + app_user_name)
    else:
        print("Error getting session token")

- Project 13 - 1.TMCI Sénégal Etude Longitudinale
  - App user 167 - Phase pilote
- Project 16 - 2.TIMCI Sénégal EPS / Etude temporelle
- Project 15 - 3.TIMCI Sénégal Etudes qualitatives
- Project 17 - 4.TIMCI Sénégal Etude de coûts
- Project 14 - Database stat tests
  - App user 185 - UCAD core team
  - App user 184 - Hospital research assistant
  - App user 172 - Research assistant
- Project 7 - Test
  - App user 115 - Helene test user
  - App user 74 - test user
- Project 3 - TIMCI cost and cost effectiveness
  - App user 128 - Data collector
  - App user 21 - Data collector
- Project 11 - TIMCI data collection test
  - App user 86 - RCT / LS data collector
  - App user 85 - CC data collector
  - App user 84 - HB data collector
  - App user 83 - FB data collector
- Project 5 - TIMCI India
- Project 9 - TIMCI Kenya
  - App user 80 - Test data collector
- Project 8 - TIMCI Myanmar
  - App user 107 - Tester
  - App user 79 - YeMinLatt
- Project 12 - TIMCI qualitative reference
  - App 

In [9]:
with open("odk_central_credentials.json") as f:
    credentials = json.load(f)
    central_url = credentials["central_url"]
    central_email = credentials["central_email"]
    central_password = credentials["central_password"]

    session_token = get_session_token(central_url, central_email, central_password)
    if session_token:
        # List all submissions for form "01-TIMCI-CRF-Facility" in project 14
        df = list_all_submissions(central_url, session_token, 14, "01-TIMCI-CRF-Facility")
        print(df.head(5))
    else:
        print("Error getting session token")

                                  instanceId  submitterId  \
0  uuid:40501623-d9eb-45a9-bd0a-a14326bde764          172   
1  uuid:026ee164-8895-4ba0-a2af-b17de6c3fc0f          172   
2  uuid:eaa1981e-955c-45a3-99b0-67317587e0df          172   
3  uuid:33773827-f67e-42f2-997d-1848e308d4fe          172   
4  uuid:e3414c44-e9d8-4859-a791-b10326d8dde1          172   

                   deviceId                 createdAt updatedAt  
0  collect:rlhNppIkdCdO9xM2  2021-03-10T21:33:08.255Z      None  
1  collect:rlhNppIkdCdO9xM2  2021-03-09T11:08:44.513Z      None  
2  collect:rlhNppIkdCdO9xM2  2021-03-09T07:47:31.359Z      None  
3  collect:rlhNppIkdCdO9xM2  2021-03-09T07:42:44.458Z      None  
4  collect:rlhNppIkdCdO9xM2  2021-02-17T13:20:26.170Z      None  


In [10]:
with open("odk_central_credentials.json") as f:
    credentials = json.load(f)
    central_url = credentials["central_url"]
    central_email = credentials["central_email"]
    central_password = credentials["central_password"]

    session_token = get_session_token(central_url, central_email, central_password)
    if session_token:
        df = data_document(central_url, session_token, 14, "01-TIMCI-CRF-Facility")
        print(df.head(5))
    else:
        print("Error getting session token")

                                        __id       today  \
0  uuid:40501623-d9eb-45a9-bd0a-a14326bde764  2021-03-10   
1  uuid:026ee164-8895-4ba0-a2af-b17de6c3fc0f  2021-03-09   
2  uuid:eaa1981e-955c-45a3-99b0-67317587e0df  2021-03-09   
3  uuid:33773827-f67e-42f2-997d-1848e308d4fe  2021-03-09   
4  uuid:e3414c44-e9d8-4859-a791-b10326d8dde1  2021-02-17   

                           start                            end  \
0  2021-03-10T22:27:18.764+01:00  2021-03-10T22:33:02.942+01:00   
1  2021-03-09T11:38:36.060+01:00  2021-03-09T12:08:37.450+01:00   
2  2021-03-09T08:46:48.819+01:00  2021-03-09T08:47:25.138+01:00   
3  2021-03-09T08:40:42.173+01:00  2021-03-09T08:42:38.056+01:00   
4  2021-02-17T14:13:28.562+01:00  2021-02-17T14:19:55.136+01:00   

                   deviceid                          screening_id  \
0  collect:rlhNppIkdCdO9xM2  d124f0ea-f882-4378-9d56-ee429b1981c9   
1  collect:rlhNppIkdCdO9xM2  dca599cd-b9dc-4aad-aeff-78999bb9ecaf   
2  collect:rlhNppIkdCdO9xM2  

In [11]:
with open("odk_central_credentials.json") as f:
    credentials = json.load(f)
    central_url = credentials["central_url"]
    central_email = credentials["central_email"]
    central_password = credentials["central_password"]
    backup_passphrase = credentials["backup_passphrase"]

    session_token = get_session_token(central_url, central_email, central_password)
    if session_token:
        # Backup of ODK Central
        backup(central_url, session_token, backup_passphrase)
        print("OK")
    else:
        print("Error getting session token")

OK
